In [7]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()
driver.get("https://www.vinted.fr/")
assert "vinted" in driver.title

elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

WebDriverException: Message: Process unexpectedly closed with status 255
